[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/tabular-classification/sklearn/fraud-detection/fraud-classifier-sklearn.ipynb)


# Fraud classification using sklearn

This notebook illustrates how sklearn models can be upladed to the Unbox platform.

In [ ]:
!curl "https://raw.githubusercontent.com/unboxai/examples-gallery/main/tabular-classification/sklearn/requirements.txt" --output "requirements.txt"

In [ ]:
!pip install -r requirements.txt

## Importing the modules and loading the dataset

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

We have stored a sample of the original dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. Alternatively, you can also find the full dataset on [this Kaggle competition](https://www.kaggle.com/datasets/kartik2112/fraud-detection?select=fraudTrain.csv). The dataset in our example corresponds to the first 10,000 lines of the original Kaggle competition dataset.

In [2]:
DATASET_URL = "https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/fraudTrainSample.csv"

In [3]:
data = pd.read_csv(DATASET_URL)

In [4]:
# Relevant columns
feature_names = ['amt', 'cc_num', 'merchant', 'category','state','job']
label = ['is_fraud']

# Outputs
class_names = ["normal", "fraudulent"]

clean_raw_data = data[feature_names + label]

In [5]:
X = clean_raw_data.drop('is_fraud', 1)
y = clean_raw_data['is_fraud']

In [6]:
X.head()

,amt,cc_num,merchant,category,state,job
0,4.97,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,NC,"Psychologist, counselling"
1,107.23,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,WA,Special educational needs teacher
2,220.11,38859492057661,fraud_Lind-Buckridge,entertainment,ID,Nature conservation officer
3,45.00,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,MT,Patent attorney
4,41.96,375534208663984,fraud_Keeling-Crist,misc_pos,VA,Dance movement psychotherapist


## Pre-processing the data and splitting it into training and validation sets

In [12]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    enc_dfs = []
    for feature, enc in encoders.items():
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        enc_dfs.append(enc_df)
    df = pd.concat([df] + enc_dfs, axis=1)
    df.drop(list(encoders.keys()), axis=1, inplace=True)
    return df

In [13]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='error')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [14]:
categorical_feature_names = ['cc_num', 'merchant', 'category', 'state', 'job']

In [15]:
encoders = create_encoder_dict(X, categorical_feature_names)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

x_val_one_hot

,amt,cc_num_60416207185,cc_num_60422928733,cc_num_60423098130,cc_num_60427851591,cc_num_60487002085,cc_num_60490596305,cc_num_60495593109,cc_num_501802953619,cc_num_501828204849,...,job_Video editor,job_Visual merchandiser,job_Volunteer coordinator,job_Warden/ranger,job_Waste management officer,job_Water engineer,job_Water quality scientist,job_Web designer,job_Wellsite geologist,job_Writer
0,1.51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,48.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,4.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,8.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,88.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,51.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Training and evaluating the model's performance

In [17]:
sklearn_model = GradientBoostingClassifier(random_state=1300)
sklearn_model.fit(x_train_one_hot, y_train)

GradientBoostingClassifier(random_state=1300)

In [18]:
print(classification_report(y_val, sklearn_model.predict(x_val_one_hot)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1988
           1       0.80      0.67      0.73        12

    accuracy                           1.00      2000
   macro avg       0.90      0.83      0.86      2000
weighted avg       1.00      1.00      1.00      2000



## Unbox part!

### pip installing unboxapi

In [ ]:
!pip install unboxapi

### Instantiating the client

In [19]:
import unboxapi

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Creating a project on the platform

In [20]:
from unboxapi.tasks import TaskType

project = client.create_project(name="Fraud classification", 
                                task_type=TaskType.TabularClassification,
                                description="Evaluation of ML approaches to detect frauds")

Created your project. Check out https://unbox.ai/projects!


### Uploading the validation set

In [21]:
# Add the ground truths to the ordinal dataset for Unbox
x_val['is_fraud'] = y_val.values
x_train['is_fraud'] = y_train.values

/var/folders/9z/j3bd32nd47j_l0thnbj6vbnw0000gn/T/ipykernel_78449/3713627219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_val['is_fraud'] = y_val.values
/var/folders/9z/j3bd32nd47j_l0thnbj6vbnw0000gn/T/ipykernel_78449/3713627219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['is_fraud'] = y_train.values


In [22]:
dataset = project.add_dataframe(
    df=x_val.sample(1000),
    class_names=class_names,
    label_column_name='is_fraud',
    commit_message='this is my fraud dataset',
    feature_names=feature_names,
    categorical_feature_names=categorical_feature_names,
)

Uploading dataset to Unbox! Check out https://unbox.ai/datasets to have a look!


### Uploading the model

First, it is important to create a `predict_proba` function, which is how Unbox interacts with your model

In [23]:
def predict_proba(model, input_features: np.ndarray, col_names, one_hot_encoder, encoders):
    """Convert the raw input_features into one-hot encoded features
    using our one hot encoder and each feature's encoder. """
    df = pd.DataFrame(input_features, columns=col_names)
    encoded_df = one_hot_encoder(df, encoders)
    return model.predict_proba(encoded_df.to_numpy())

Let's test the `predict_proba` function to make sure the input-output format is consistent with what Unbox expects:

In [24]:
# Test the predict function
predict_proba(sklearn_model, x_val[:3][feature_names].to_numpy(), feature_names, data_encode_one_hot, encoders)

array([[9.99998752e-01, 1.24800170e-06],
       [9.99998752e-01, 1.24800170e-06],
       [9.99998752e-01, 1.24800170e-06]])

Now, we can upload the model:

In [25]:
from unboxapi.models import ModelType

model = project.add_model(
    function=predict_proba, 
    model=sklearn_model,
    model_type=ModelType.sklearn,
    class_names=class_names,
    name='Fraud detection',
    commit_message='this is my fraud classification model',
    feature_names=feature_names,
    train_sample_df=x_train,
    train_sample_label_column_name='is_fraud',
    categorical_feature_names=categorical_feature_names,
    col_names=feature_names,
    requirements_txt_file='requirements.txt',
    one_hot_encoder=data_encode_one_hot,
    encoders=encoders,
)

Bundling model and artifacts...
Uploading model to Unbox! Check out https://unbox.ai/models to have a look!


/Users/gustavocid/miniconda3/envs/unbox-examples/lib/python3.8/site-packages/joblib/numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
